In [7]:
import pandas as pd
import numpy as np
import ta
import xgboost as xgb
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import requests
import feedparser


In [8]:

#Fetching of data from serveral sources and merging

oanda_api_key = "b9846701e3d5d9b2d73127d5040aac28-5d86be73abbdec141a31891cb532d340"
oanda_url = "https://api-fxpractice.oanda.com/v3/instruments/EUR_USD/candles?granularity=M30"
headers = {"Authorization": f"Bearer {oanda_api_key}"}
response = requests.get(oanda_url, headers=headers).json()

forex_data = pd.DataFrame(response['candles'])


if 'mid' in forex_data.columns:
    forex_data['open'] = forex_data['mid'].apply(lambda x: float(x['o']))
    forex_data['high'] = forex_data['mid'].apply(lambda x: float(x['h']))
    forex_data['low'] = forex_data['mid'].apply(lambda x: float(x['l']))
    forex_data['close'] = forex_data['mid'].apply(lambda x: float(x['c']))
    forex_data.drop(columns=['mid'], inplace=True)
else:
    print("'mid' column missing from forex_data!")


forex_data['time'] = pd.to_datetime(forex_data['time']).dt.tz_localize(None)


forex_data['date_time'] = forex_data['time'].dt.floor('D')  
forex_data['rounded_time'] = forex_data['time'].dt.floor('h')  

fred_api_key = "5de83463f5e12c01d110a2322a075ecc"
fred_series = {
    "inflation": "CPIAUCSL",
    "gdp": "GDPC1",
    "unemployment": "UNRATE",
    "interest_rate": "DGS10"
}

def get_fred_data(series_id):
    url = f"https://api.stlouisfed.org/fred/series/observations?series_id={series_id}&api_key={fred_api_key}&file_type=json"
    response = requests.get(url).json()

    if 'observations' not in response:
        print(f"Failed to fetch {series_id}")
        return pd.DataFrame()
    
    data = pd.DataFrame(response['observations'])
    data['date'] = pd.to_datetime(data['date'])
    data[series_id] = pd.to_numeric(data['value'], errors='coerce')
    return data[['date', series_id]]


fred_data = pd.DataFrame()
for label, series_id in fred_series.items():
    indicator_df = get_fred_data(series_id)
    if indicator_df.empty:
        continue
    if fred_data.empty:
        fred_data = indicator_df
    else:
        fred_data = fred_data.merge(indicator_df, on='date', how='outer')

fred_data.sort_values('date', inplace=True)
fred_data.ffill(inplace=True) 


forex_data.sort_values('date_time', inplace=True)
merged_data = pd.merge_asof(
    forex_data,
    fred_data,
    left_on='date_time',
    right_on='date',
    direction='backward'
)

feed_url = "https://feeds.bbci.co.uk/news/business/rss.xml"
feed = feedparser.parse(feed_url)

news_df = pd.DataFrame({
    'headline': [entry.title for entry in feed.entries],
    'datetime': pd.to_datetime([entry.published for entry in feed.entries], errors='coerce')
})

keywords = ['interest rate', 'inflation', 'central bank', 'monetary policy']
news_df = news_df[news_df['headline'].str.contains('|'.join(keywords), case=False, na=False)]

news_df['rounded_time'] = news_df['datetime'].dt.floor('h')  
news_grouped = news_df.groupby('rounded_time').agg({'headline': lambda x: ' | '.join(x)}).reset_index()
merged_data = merged_data.merge(news_grouped, on='rounded_time', how='left')


merged_data = merged_data.drop(columns=['date_time','date','rounded_time'])

merged_data = merged_data.sort_values(by='time',ascending=True)
merged_data


,complete,volume,time,open,high,low,close,CPIAUCSL,GDPC1,UNRATE,DGS10,headline
0,True,16954,2025-04-04 11:00:00,1.10305,1.10653,1.10222,1.10503,319.615,23542.349,4.2,4.01,NaN
19,True,19733,2025-04-04 11:30:00,1.10500,1.10800,1.10454,1.10603,319.615,23542.349,4.2,4.01,NaN
18,True,20256,2025-04-04 12:00:00,1.10604,1.10890,1.10430,1.10838,319.615,23542.349,4.2,4.01,NaN
17,True,28198,2025-04-04 12:30:00,1.10833,1.10898,1.10076,1.10200,319.615,23542.349,4.2,4.01,NaN
16,True,18771,2025-04-04 13:00:00,1.10203,1.10481,1.10139,1.10287,319.615,23542.349,4.2,4.01,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
472,True,1664,2025-04-18 18:30:00,1.13940,1.13956,1.13914,1.13943,319.615,23542.349,4.2,4.34,NaN
473,True,611,2025-04-18 19:00:00,1.13941,1.13961,1.13934,1.13960,319.615,23542.349,4.2,4.34,NaN
474,True,560,2025-04-18 19:30:00,1.13960,1.13978,1.13934,1.13952,319.615,23542.349,4.2,4.34,NaN
480,True,364,2025-04-18 20:00:00,1.13954,1.13954,1.13916,1.13944,319.615,23542.349,4.2,4.34,NaN


In [9]:



# forex_data['sma_20'] = ta.trend.sma_indicator(forex_data['close'], window=20) 

# forex_data['macd_histogram'] = ta.trend.macd_diff(forex_data['close'])  
# forex_data['bollinger_middle'] = ta.volatility.bollinger_mavg(forex_data['close'], window=20)  
# forex_data['bollinger_upper'] = ta.volatility.bollinger_hband(forex_data['close'], window=20) 
# forex_data['bollinger_lower'] = ta.volatility.bollinger_lband(forex_data['close'], window=20) 


# forex_data['atr'] = ta.volatility.average_true_range(forex_data['high'], forex_data['low'], forex_data['close'], window=14)


# forex_data['stochastic_k'] = ta.momentum.stoch(forex_data['high'], forex_data['low'], forex_data['close'], window=14)
# forex_data['stochastic_d'] = ta.momentum.stoch_signal(forex_data['high'], forex_data['low'], forex_data['close'], window=14)


# forex_data['obv'] = ta.volume.on_balance_volume(forex_data['close'], forex_data['volume'])


# forex_data['roc'] = ta.momentum.roc(forex_data['close'], window=10)


# forex_data['vroc'] = ta.momentum.roc(forex_data['volume'], window=10)






# forex_data['candle_range'] = forex_data['high'] - forex_data['low']


# forex_data['body_size'] = abs(forex_data['close'] - forex_data['open'])


# forex_data['upper_shadow'] = forex_data['high'] - pd.DataFrame([forex_data['close'], forex_data['open']]).max()


# forex_data['lower_shadow'] = pd.DataFrame([forex_data['close'], forex_data['open']]).min() - forex_data['low']

merged_data['price_pct_change'] = (merged_data['close'] - merged_data['open']) / merged_data['open'] * 100

merged_data['price_change'] = merged_data['close'] - merged_data['open']

merged_data['deviation_pts'] = merged_data['price_pct_change']  
merged_data['ema_20'] = ta.trend.ema_indicator(merged_data['close'], window=20) 

merged_data['macd'] = ta.trend.macd(merged_data['close'])  
merged_data['macd_signal'] = ta.trend.macd_signal(merged_data['close'])  
merged_data['rsi'] = ta.momentum.rsi(merged_data['close'], window=14)  
merged_data["time"] = pd.to_datetime(merged_data["time"])
merged_data.set_index("time", inplace=True)

In [10]:

X_train = merged_data.loc[:"2025-04-14"] 
X_val = merged_data.loc["2025-04-14":"2025-04-16"] 
X_test = merged_data.loc["2025-04-16":]

y_train = np.where(X_train["macd"] > X_train["macd_signal"], 1,0)
y_val = np.where(X_val["macd"] > X_val["macd_signal"], 1, 0) 
y_test = np.where(X_test["macd"] > X_test["macd_signal"], 1,0)

In [11]:

print("Unique values in y_train:", np.unique(y_train))
print("Unique values in y_val:", np.unique(y_val))
print("Unique values in y_train:", np.unique(y_test))

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)
# xgb_model = xgb.XGBClassifier(
#     n_estimators=100,
#     max_depth=5,
#     learning_rate=0.1,
#     objective="binary:logistic" 
# )
xgb_model = xgb.XGBClassifier(**grid_search.best_params_)
xgb_model.fit(X_train, y_train,eval_set=[(X_val, y_val)], verbose=True)
# xgb_model.fit(X_train, y_train, )
y_pred = xgb_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy:.2f}")


Unique values in y_train: [0 1]
Unique values in y_val: [0 1]
Unique values in y_train: [0 1]


/Users/redietmebrat/Library/Python/3.9/lib/python/site-packages/sklearn/utils/extmath.py:1101: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/Users/redietmebrat/Library/Python/3.9/lib/python/site-packages/sklearn/utils/extmath.py:1106: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
/Users/redietmebrat/Library/Python/3.9/lib/python/site-packages/sklearn/utils/extmath.py:1126: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count


ValueError: could not convert string to float: 'Falling petrol prices push UK inflation down | Falling petrol prices push UK inflation down'

In [29]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    "n_estimators": [50, 100, 200],
    "max_depth": [3, 5, 7],
    "learning_rate": [0.01, 0.1, 0.2],
    "subsample": [0.5, 0.8, 1]
}

grid_search = GridSearchCV(xgb.XGBClassifier(objective="binary:logistic"),
                           param_grid, cv=3, scoring="accuracy", verbose=2)
grid_search.fit(X_train, y_train)
print("Best Hyperparameters:", grid_search.best_params_)


Fitting 3 folds for each of 81 candidates, totalling 243 fits
[CV] END learning_rate=0.01, max_depth=3, n_estimators=50, subsample=0.5; total time=   0.0s
[CV] END learning_rate=0.01, max_depth=3, n_estimators=50, subsample=0.5; total time=   0.0s
[CV] END learning_rate=0.01, max_depth=3, n_estimators=50, subsample=0.5; total time=   0.0s
[CV] END learning_rate=0.01, max_depth=3, n_estimators=50, subsample=0.8; total time=   0.0s
[CV] END learning_rate=0.01, max_depth=3, n_estimators=50, subsample=0.8; total time=   0.0s
[CV] END learning_rate=0.01, max_depth=3, n_estimators=50, subsample=0.8; total time=   0.0s
[CV] END learning_rate=0.01, max_depth=3, n_estimators=50, subsample=1; total time=   0.0s
[CV] END learning_rate=0.01, max_depth=3, n_estimators=50, subsample=1; total time=   0.0s
[CV] END learning_rate=0.01, max_depth=3, n_estimators=50, subsample=1; total time=   0.0s
[CV] END learning_rate=0.01, max_depth=3, n_estimators=100, subsample=0.5; total time=   0.0s
[CV] END lear

In [30]:
xgb_model = xgb.XGBClassifier(**grid_search.best_params_)
xgb_model.fit(X_train, y_train,eval_set=[(X_val, y_val)], verbose=True)
# xgb_model.fit(X_train, y_train, )
y_pred = xgb_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy:.2f}")


[0]	validation_0-logloss:0.66685
[1]	validation_0-logloss:0.63461
[2]	validation_0-logloss:0.63010
[3]	validation_0-logloss:0.60230
[4]	validation_0-logloss:0.58152
[5]	validation_0-logloss:0.57036
[6]	validation_0-logloss:0.56172
[7]	validation_0-logloss:0.54848
[8]	validation_0-logloss:0.53316
[9]	validation_0-logloss:0.52760
[10]	validation_0-logloss:0.51184
[11]	validation_0-logloss:0.50547
[12]	validation_0-logloss:0.50505
[13]	validation_0-logloss:0.50372
[14]	validation_0-logloss:0.49958
[15]	validation_0-logloss:0.48897
[16]	validation_0-logloss:0.48213
[17]	validation_0-logloss:0.47788
[18]	validation_0-logloss:0.47103
[19]	validation_0-logloss:0.47231
[20]	validation_0-logloss:0.47223
[21]	validation_0-logloss:0.47906
[22]	validation_0-logloss:0.47444
[23]	validation_0-logloss:0.47043
[24]	validation_0-logloss:0.47182
[25]	validation_0-logloss:0.46892
[26]	validation_0-logloss:0.47780
[27]	validation_0-logloss:0.47199
[28]	validation_0-logloss:0.47086
[29]	validation_0-loglos